In [114]:
def tocsv_hybrid(uid):
    import pickle
    import csv
    with open('student_paper_mappings.pkl', 'rb') as f:
        data_ref = pickle.load(f)
        matrix=data_ref.as_matrix()
        for index,i in enumerate(matrix):
            matrix[index][0]=matrix[index][0].split(',')

        arr=[]
        for (index,it1) in enumerate(matrix):
            arr.append([])
            for it2 in matrix:

                it1_set = set(it1[0])
                it2_set = set(it2[0])
                arr[index].append(it1_set & it2_set)
        # set (i,i ) to empty set before finding most matched user
        for (index,it) in enumerate(arr):
            arr[index][index]=set()
        # Arrays to maintain the most matched user, and no. of common papers
        match=[0]*120
        count=[0]*120
        # for i, val in enumerate(arr):
        #     print(i, val)
        for (index1,it2) in enumerate(arr):
            #print(it2)
            for(index2,it2) in enumerate(it2):
                #print(index1,index2)
                a = len(arr[index1][index2])
                #print(len(arr[index1][index2]))
                if(count[index1]<len(arr[index1][index2])):
                    match[index1]=index2
                    count[index1]=len(arr[index1][index2])
        for (index,i) in enumerate(match):
            s=(set(matrix[i][0])-set(matrix[index][0]))
        l=list(s)
        cleanedList = [x for x in l if str(x) != 'nan']
        cl = [int(i) for i in cleanedList]

        with open('papers.pkl', 'rb') as f:
            data_papers = pickle.load(f)
        uid=list(data_ref.student_id).index(uid)
        s=(set(matrix[match[uid]][0])-set(matrix[uid][0]))
        l=list(s)
        cleanedList = [x for x in l if str(x) != 'nan']
        cl = [int(i) for i in cleanedList]
        print(cl)
        list_papers=[]
        with open('user_rec.csv','w') as csvfile:
            fieldnames=['paper_id','title','category']
            writer=csv.DictWriter(csvfile,fieldnames=fieldnames)
            writer.writeheader()
            for i in cl:
                writer.writerow({'paper_id':i,'title':data_papers.title[i],'category':data_papers.arxiv_primary_category[i]})
                list_papers.append(i)
                #print("User ID ",uid)
                print('paper_id ',i,'title ',data_papers.title[i],'category ',data_papers.arxiv_primary_category[i])
            
            
tocsv_hybrid(621741617)



[11909, 10724, 8809, 8312, 8844, 9784, 8423, 8823, 12333, 9503, 10156]
paper_id  11909 title  The Wall and The Ball: A Study of Domain Referent Spreadsheet Errors category  HC
paper_id  10724 title  CrashScope: A Practical Tool for Automated Testing of Android
  Applications category  SE
paper_id  8809 title  Examining the Impact of Platform Properties on Quality Attributes category  SE
paper_id  8312 title  EuSpRIG 2006 Commercial Spreadsheet Review category  SE
paper_id  8844 title  Ontology for Mobile Phone Operating Systems category  SE
paper_id  9784 title  Service-Oriented Architecture in Industrial Automation Systems - The
  case of IEC 61499: A Review category  SE
paper_id  8423 title  Software Components for Web Services category  SE
paper_id  8823 title  Towards a better understanding of testing if conditionals category  SE
paper_id  12333 title  HTTP Mailbox - Asynchronous RESTful Communication category  SE
paper_id  9503 title  Hierarchical Variability Modeling for Software

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [116]:
def content(papers,uid):
    import pickle
    import pandas as pd

    with open('papers.pkl', 'rb') as f :
        data = pickle.load(f)
    with open('student_paper_mappings.pkl', 'rb') as f:
        data_ref = pickle.load(f)
        matrix=data_ref.as_matrix()
        for index,i in enumerate(matrix):
            matrix[index][0]=matrix[index][0].split(',')
    uid=list(data_ref.student_id).index(uid)
    mat=data.as_matrix()
    user_summar=[]
    for i in matrix[uid][0]:
        user_summar.append(mat[int(i)-1][5])
       
        
    
    summar=[]
    for i in papers:
        summar.append(mat[int(i)-1][5])
    print(user_summar) 
    print(summar)
    summar_new = pd.DataFrame({'summary':summar})
    print(summar_new.head())
    summary=summar_new["summary"]
    
    user_summar_new = pd.DataFrame({'user_summary':user_summar})
    print(user_summar_new.head())
    user_summary=user_summar_new["user_summary"]
    import nltk
    from nltk.corpus import stopwords
    set(stopwords.words('english'))
    import string
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    porter_stemmer = PorterStemmer()
    print(summary)
    def tokenize_summary(summary):
        summ = [char for char in summary if char not in string.punctuation]
        summ = ''.join(summ)
    
        word = [word for word in summ.split() if word.lower() not in stopwords.words('english')]
        return[porter_stemmer.stem(word) for word in word]
    print(tokenize_summary(summary[0]))
    
    summary_tokens = []
    for i in summary:
            summary_tokens.append(tokenize_summary(i))
    
    user_summary_tokens = []
    for i in user_summary:
            user_summary_tokens.append(tokenize_summary(i))
            
            
    print(user_summary_tokens[1])
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc,lowercase=False,stop_words='english', use_idf=False, norm='l1')
    Y = vectorizer.fit_transform(summary_tokens)
    X = vectorizer.transform(user_summary_tokens)
    X=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    Y= pd.DataFrame(Y.toarray(), columns=vectorizer.get_feature_names())
    X=X.as_matrix()
    Y=Y.as_matrix()
    from sklearn.metrics.pairwise import cosine_similarity
    for index,i in enumerate(X):
        similarity = (cosine_similarity(X[index:index+1], Y))
        flattened_list = [y for x in similarity for y in x]
        similarity_sorted=sorted(flattened_list,reverse=True)
        similarity_topn = similarity_sorted[1:11]
    return similarity_topn

content([11909, 10724,  8809,  8312,  8844,  9784,  8423,  8823, 12333,  9503,
 10156],621741617)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


['There has been a noticeable shift in the relative composition of the industry\nin the developed countries in recent years; manufacturing is decreasing while\nthe service sector is becoming more important. However, currently most\nsimulation models for investigating service systems are still built in the same\nway as manufacturing simulation models, using a process-oriented world view,\ni.e. they model the flow of passive entities through a system. These kinds of\nmodels allow studying aspects of operational management but are not well suited\nfor studying the dynamics that appear in service systems due to human\nbehaviour. For these kinds of studies we require tools that allow modelling the\nsystem and entities using an object-oriented world view, where intelligent\nobjects serve as abstract "actors" that are goal directed and can behave\nproactively. In our work we combine process-oriented discrete event simulation\nmodelling and object-oriented agent based simulation modelling to i

['basic', 'aim', 'studi', 'give', 'possibl', 'model', 'handl', 'uncertain', 'inform', 'model', 'work', 'framework', 'datalog', 'first', 'concept', 'fuzzi', 'datalog', 'summar', 'extens', 'intuitionist', 'intervalvalu', 'fuzzi', 'logic', 'given', 'concept', 'bipolar', 'fuzzi', 'datalog', 'introduc', 'base', 'idea', 'concept', 'multivalu', 'knowledgebas', 'defin', 'quadrupl', 'background', 'knowledg', 'deduct', 'mechan', 'connect', 'algorithm', 'function', 'set', 'program', 'help', 'us', 'determin', 'uncertainti', 'level', 'result', 'last', 'possibl', 'evalu', 'strategi', 'given']


/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[0.08202216692278198,
 0.06532181680580171,
 0.06187622439394684,
 0.058321184351980436,
 0.057827625257403106,
 0.04067819981796425,
 0.040177170184515394,
 0.03575421226192905,
 0.019676044536698512,
 0.0]